In [1]:
"""

Prepare zmin and zmax to be incorporated in the sub-shelf melt file 

author: Clara Burgard

"""

'\n\nPrepare zmin and zmax to be incorporated in the sub-shelf melt file \n\nauthor: Clara Burgard\n\n'

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [4]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_Schmidt = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/'

In [5]:
file_Justine_orig = xr.open_dataset(inputpath_raw + 'Mask_Iceshelf_IMBIE2_v2.nc')
mask_Justine_05 = xr.open_dataset(inputpath_Schmidt + 'Iceshelf_extrap_05degrees.nc')

In [6]:
file_Justine_withzmin_zmax = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')


In [7]:
ID_list = {}
for bb in file_Justine_orig['Subregions'].values:
    ID_list[bb] = []
    for kisf in file_Justine_orig['ID'].values:
        if file_Justine_orig['Subregions'].sel(ID=kisf) == bb:
            ID_list[bb].append(kisf)

In [11]:
zmin_list = []
zmax_list = []

for id_basin, bb in enumerate(tqdm(ID_list.keys())): 
    # identify minimum zmin and maximum zmax in each basin
    zmin_basin = file_Justine_withzmin_zmax['z_perc01'].sel(ID=ID_list[bb]).min('ID')
    zmax_basin = file_Justine_withzmin_zmax['z_perc99'].sel(ID=ID_list[bb]).max('ID')
    zmin_list.append(zmin_basin.assign_coords({'region': id_basin+1}))
    zmax_list.append(zmax_basin.assign_coords({'region': id_basin+1}))

zmin_all = xr.concat(zmin_list, dim='region')
zmax_all = xr.concat(zmax_list, dim='region')

  0%|          | 0/18 [00:00<?, ?it/s]

In [13]:
zmin_all.drop('quantile')

<xarray.DataArray 'z_perc01' (region: 18)>
array([ 47.26678986,  31.01032006, 158.85066554,  17.38851814,
        27.84002133, 120.76192289,   4.02382585,  52.49395629,
        13.20011111,   8.10292524,  50.48477528,   1.        ,
         4.0984235 ,  17.52245875,   1.66605822, 204.66308665,
       350.00383846,  13.81547664])
Coordinates:
  * region   (region) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18

In [16]:
z_info = xr.Dataset()
z_info['min_injection_depth'] = zmin_all.drop('quantile')
z_info['min_injection_depth'].attrs['standard_name'] = 'Minimal depth where to inject the sub-ice-shelf meltwater'
z_info['min_injection_depth'].attrs['units'] = 'm below sea-level'
z_info['min_injection_depth'].attrs['method'] = 'This is the basin-wide shallowest point (1st percentile) of the ice-shelf draft '
z_info['max_injection_depth'] = zmax_all.drop('quantile')
z_info['max_injection_depth'].attrs['standard_name'] = 'Maximal depth where to inject the sub-ice-shelf meltwater'
z_info['max_injection_depth'].attrs['units'] = 'm below sea-level'
z_info['max_injection_depth'].attrs['method'] = 'This is the basin-wide deepest bathymetry (99th percentile)'

In [18]:
z_info.attrs['description'] = 'The geometric properties inferred from below the ice-shelf cavities are based on a BedMachine3 (Morlighem 2022) version downscaled to 2 km resolution.'
z_info.attrs['reference'] = 'Morlighem, M. (2022). MEaSUREs BedMachine Antarctica. (NSIDC-0756, Version 3). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. doi: 10.5067/FPSU0V1MWUB6. Date Accessed 05-15-2025.'
z_info.attrs['creation'] = 'C. Burgard on Tue Jun 17 18:20 2025'

In [20]:
z_info.to_netcdf(inputpath_Schmidt + 'injection_depths_subshelf_melt.nc')

Check output

In [89]:
all_info = xr.open_dataset(inputpath_Schmidt + 'subshelf_meltwater_injection_weights_and_depths.nc')

In [95]:
all_info['z_min'].plot(vmax=10)

In [98]:
all_info['z_min'].plot()